In [44]:
import numpy as np
import math

In [45]:
class DataLoader():
    
    def __init__(self,data_path):
        self.data = np.load(data_path)
    
    def train_test_split(self):
        
        x_train = self.data['train_images'].reshape(self.data['train_images'].shape[0],-1)
        x_test = self.data['test_images'].reshape(self.data['test_images'].shape[0],-1)
        x_val = self.data['val_images'].reshape(self.data['val_images'].shape[0],-1)
        y_train = self.data['train_labels']
        y_test = self.data['test_labels']
        y_val = self.data['val_labels']
        
        return x_train,y_train,x_test,y_test,x_val,y_val
    

In [46]:
class Multiclass_Gaussian_Naive_Bayes():
    
    def __init__(self,n_features,n_classes):
        
        self.n_classes = n_classes
        self.n_features = n_features
        self.mean = np.zeros((n_classes,n_features))
        self.variance = np.zeros((n_classes,n_features))
        self.eps = 1e-7
    def fit(self,X,counts):
    
        for i in range(np.size(counts)):
            if i==0:
                st_idx = 0
            else:
                st_idx = np.sum(counts[0:i])
            curr_X = X[st_idx:st_idx+counts[i]]
            self.mean[i] = np.mean(curr_X,axis = 0)
            diff = curr_X - self.mean[i]
            self.variance[i] = np.mean(np.square(diff),axis = 0)

    def gaussian(self,X, mu, var):
        
        return 1 / ((2 * np.pi) ** (1 / 2) * var ** 0.5) * np.exp(-0.5 * (X-mu)**2/var)

    def predict(self,x_test):
        
        pred = np.zeros(x_test.shape[0])
        
        for j in range(x_test.shape[0]):
            best_class = 0
            best_likelihood = -math.inf
            for i in range(self.n_classes):
                likelihood = self.gaussian(x_test[j],self.mean[i],self.variance[i])
                log_likelihood = np.sum(np.log(likelihood+self.eps))
                if best_likelihood < log_likelihood:
                    best_likelihood = log_likelihood
                    best_class = i
            pred[j] = best_class
        return pred
                
    def accuracy(self,pred,actual):
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        return accuracy

In [47]:
class LDA():
    
    def __init__(self,n_features,n_classes):
        self.n_features = n_features
        self.n_classes = n_classes
        self.S_w = np.zeros((n_features,n_features))
        self.S_b = np.zeros((n_features,n_features))
        self.mu = np.zeros(n_features)
        self.mu_class = np.zeros((self.n_classes,self.n_features))

    def fit(self,X,counts):
        self.mu = np.mean(X,axis = 0)
        self.mu_class[0] = np.mean(X[0:counts[0],:],axis = 0)
        self.mu_class[1] = np.mean(X[counts[0]:,:],axis = 0)
        self.S_w = np.dot((X[0:counts[0],:]-self.mu_class[0]).T,X[0:counts[0],:]-self.mu_class[0]) + np.dot((X[counts[0]:,:]-self.mu_class[1]).T,X[counts[0]:,:]-self.mu_class[1])
        self.S_b = np.dot((self.mu_class[0] - self.mu_class[1]).T,self.mu_class[0] - self.mu_class[1])
            
    def predict(self,x_test):
        
        v = np.dot(np.linalg.inv(self.S_w),self.mu_class[0] - self.mu_class[1])
        pred = [1 if abs(np.dot(x_test[i],v)-np.dot(self.mu_class[0],v))> abs(np.dot(x_test[i],v)-np.dot(self.mu_class[1],v)) else 0 for i in range(x_test.shape[0])]
        return pred
    
    def accuracy(self,pred,actual):
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        return accuracy

In [48]:
class LogisticRegression():

    def __init__(self, learning_rate=0.001):
        self.param = None
        self.learning_rate = learning_rate
        self.training_errors = []
        self.eps = 1e-7

    def initialize_parameters(self, X):
        n_features = np.shape(X)[1]
        self.param = np.ones((n_features,1))
    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))

    def fit(self, X, y, n_iterations=4000):
        X = np.insert(X,0,1,axis = 1)
        y = y.reshape(-1,1)
        self.initialize_parameters(X)
        for i in range(n_iterations):
            
            y_pred = self.sigmoid(np.dot(X,self.param))
            self.training_errors.append(np.dot(y.T,np.log(y_pred + self.eps)).item())
            self.param -= self.learning_rate * -np.dot(X.T,y - y_pred)
        
    def predict(self, X):
        
        X = np.insert(X,0,1,axis = 1)
        y_pred = np.round(self.sigmoid(np.dot(X,self.param))).astype(int)
        
        return y_pred
    
    def accuracy(self,pred,actual):
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        return accuracy

In [49]:
from scipy.special import softmax

class Multiclass_Logistic_Regression():
    
    def __init__(self, learning_rate,n_classes):
        self.param = None
        self.lr = learning_rate
        self.training_errors = []
        self.eps = 1e-7
        self.n_classes = n_classes

    def initialize_parameters(self, X):
        n_features = np.shape(X)[1]
        self.param = np.ones((self.n_classes,n_features))
    
    def one_hot(self,y):

        return np.eye(self.n_classes)[y.reshape(-1)]
    '''
    def softmax(self,probs):
        probs = probs - (np.mean(probs,axis=1)).reshape(-1,1)   ## normalization of probs
        return np.exp(probs)/(np.sum(np.exp(probs),axis = 1) + self.eps).reshape(-1,1)
    '''

    def fit(self, X, y, n_iterations=1000):
        
        X = np.insert(X,0,1,axis=1)
        y = self.one_hot(y)
        self.initialize_parameters(X)
        loss_per_iter = []
        for i in range(n_iterations):
            y_pred = softmax(np.dot(X,self.param.T),axis = 1)
            loss = -1*np.mean(y*np.log(y_pred + self.eps))
            loss_per_iter.append(loss)
            grad = np.dot((y_pred-y).T,X)
            self.param = self.param - self.lr*grad
    
    def predict(self, X):
        
        X = np.insert(X,0,1,axis=1)
        y_pred = np.argmax(softmax(np.dot(X,self.param.T),axis=1),axis = 1)
        
        return y_pred
    
    def accuracy(self,pred,actual):
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        return accuracy

In [50]:
class GaussianMLE():
    
    def __init__(self,n_classes):
        self.n_classes = n_classes
        self.eps = 1e-7
    
    def fit(self,X,counts):
        
        self.mu = np.zeros((self.n_classes,X.shape[1]))
        self.cov = np.zeros((self.n_classes,X.shape[1],X.shape[1]))
        
        for i in range(len(counts)):
            if i==0:
                st_idx = 0
            else:
                st_idx = np.sum(counts[0:i])
            curr_X = X[st_idx:st_idx+counts[i]]
            self.mu[i] = np.mean(curr_X,axis = 0)
            self.cov[i] = np.dot((curr_X-self.mu[i]).T,curr_X-self.mu[i])
            
    def log_likelihood(self,X, mu, cov):
        
        sign,log_det = np.linalg.slogdet(cov)
        det = sign*np.exp(log_det)
        return 0.5*log_det -0.5*np.dot(np.dot((X-mu).T,np.linalg.inv(cov)),X-mu)

    def predict(self,x_test):
        
        pred = np.zeros(x_test.shape[0])
        for i in range(x_test.shape[0]):
            best_class = 0
            best_likelihood = -math.inf
            for j in range(self.n_classes):
                log_likelihood = self.log_likelihood(x_test[i],self.mu[j],self.cov[j])
                if best_likelihood < log_likelihood:
                    best_likelihood = log_likelihood
                    best_class = j
            pred[i] = best_class
        return pred
    
    def accuracy(self,pred,actual):
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        return accuracy

In [51]:
class KNearestNeighbor():
    
    def __init__(self,x_train,y_train,K):
        self.k = K
        self.X = x_train
        self.y = y_train
    
    def get_Euclidean_distance(self,x):
        
        return np.sqrt(np.sum((self.X - x)**2,axis=1))
        
    def predict(self,x_test):
        
        pred = np.zeros(x_test.shape[0])
        for i in range(x_test.shape[0]):
            dist = self.get_Euclidean_distance(x_test[i])
            nearest_neighbors = dist.argsort()[0:self.k]
            unique,counts = np.unique(self.y[nearest_neighbors], return_counts = True)
            pred[i] = unique[np.argmax(counts)]
            
        return pred
    
    def accuracy(self,pred,actual):
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        return accuracy

In [62]:
class GMM():
    
    def __init__(self,n_clusters):
        self.n_clusters = n_clusters
        self.clusters = []
        self.eps = 1e-7
        
    def gaussian(self,X, mu, cov):
        n = X.shape[1]
        scaled_probs = np.zeros((X.shape[0],1))
        sign,det = np.linalg.slogdet(cov)
        for i in range(X.shape[0]):
            diff = (X[i] - mu).reshape(-1,1)
            scaled_probs[i] = np.exp(0.5*(det - np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff)))
        return scaled_probs
        
    def initialize_clusters(self,n_features):
        
        self.n_features = n_features
        for i in range(self.n_clusters):
            self.clusters.append({
                'pi_k': 1.0 / self.n_clusters,
                'mu_k': np.ones(self.n_features),
                'cov_k': np.identity(self.n_features, dtype=np.float64)
            })
    
    def expectation(self,X):
        totals = np.zeros((X.shape[0], 1), dtype=np.float64)
        for cluster in self.clusters:
        
            probs = (cluster['pi_k'] * self.gaussian(X, cluster['mu_k'], cluster['cov_k'])).astype(np.float64)

            for i in range(X.shape[0]):
                totals[i] += probs[i]

            cluster['probs'] = probs
            cluster['totals'] = totals


        for cluster in self.clusters:
            cluster['probs'] = cluster['probs']/(cluster['totals'] + self.eps)
            
    def maximization(self,X,n_epochs):
        
        N = float(X.shape[0])
        for cluster in self.clusters:
            probs = cluster['probs']
            cov_k = np.zeros((X.shape[1], X.shape[1]))
            N_k = np.sum(cluster['probs'])

            pi_k = N_k / N
            mu_k = np.sum(cluster['probs'] * X, axis=0) / (N_k + self.eps)

            for j in range(X.shape[0]):
                diff = (X[j] - mu_k).reshape(-1, 1)
                cov_k += probs[j] * np.dot(diff, diff.T)

            cov_k = cov_k/(N_k + self.eps)

            cluster['pi_k'] = pi_k
            cluster['mu_k'] = mu_k
            cluster['cov_k'] = cov_k
            
        
    def get_likelihood(self,X):

        sample_likelihoods = np.log(np.array([cluster['totals'] for cluster in self.clusters]) + self.eps)
        return np.sum(sample_likelihoods), sample_likelihoods
    
    def train_gmm(self,X,n_epochs):
    
        self.initialize_clusters(n_features=X.shape[1])
        likelihoods = np.zeros((n_epochs, ))
        scores = np.zeros((X.shape[0], self.n_clusters))

        for i in range(n_epochs):

            self.expectation(X)
            self.maximization(X,n_epochs)

            likelihood, sample_likelihoods = self.get_likelihood(X)
            likelihoods[i] = likelihood

            #print('Epoch: ', i + 1, 'Likelihood: ', likelihood)

        for i, cluster in enumerate(self.clusters):
            scores[:, i] = np.log(cluster['probs']).reshape(-1)

        return self.clusters, likelihoods, scores, sample_likelihoods

In [63]:
class GMM_classification():
    
    def __init__(self,n_classes,n_clusters):
        self.n_classes = n_classes
        self.n_clusters = n_clusters
    
    def fit(self,X,counts):
        
        self.class_conditional_densities = []
        for i in range(len(counts)):
            if i==0:
                st_idx = 0
            else:
                st_idx = np.sum(counts[0:i])
            curr_X = X[st_idx:st_idx+counts[i]]
            clf = GMM(self.n_clusters)
            
            clusters = clf.train_gmm(curr_X,100)[0]
            self.class_conditional_densities.append(clusters)
            
    def log_likelihood(self,X, clusters):
        
        for cluster in clusters:
            
        #det = sign*np.exp(log_det)
        return 0.5*log_det -0.5*np.dot(np.dot((X-mu).T,np.linalg.inv(cov)),X-mu)

    def predict(self,x_test):
        
        pred = np.zeros(x_test.shape[0])
        for i in range(x_test.shape[0]):
            best_class = 0
            best_likelihood = -math.inf
            for j in range(self.n_classes):
                mu = self.class_conditional_densities[j]['mu_k']
                cov = self.class_conditional_densities[j]['cov_k']
                log_likelihood = self.log_likelihood(x_test[i],clusters)
                if best_likelihood < log_likelihood:
                    best_likelihood = log_likelihood
                    best_class = j
            pred[i] = best_class
        return pred
    
    def accuracy(self,pred,actual):
        
        n_correct_preds = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                n_correct_preds += 1
        accuracy = n_correct_preds/actual.shape[0]
        
        return accuracy

In [ ]:
if __name__ == '__main__':

    dataset = DataLoader('../Assignment 1/pneumoniamnist.npz')
    x_train,y_train,x_test,y_test,x_val,y_val = dataset.train_test_split()
    #x_train = dataset.normalize_input(x_train)
    unique,counts = np.unique(y_train,return_counts= True)
    X = np.zeros(x_train.shape)
    y = np.zeros(y_train.shape[0],dtype = int)
    idx = 0
    for label in range(2):
        labels = np.where(y_train==label)[0]
        for i in range(np.size(labels)):
            X[idx] = x_train[labels[i]]
            y[idx] = label
            idx += 1
    
    
    clf = GaussianMLE(n_classes=2)
    clf.fit(X,counts)
    pred = clf.predict(x_test)
    accuracy = clf.accuracy(pred,y_test)
    print(f'accuracy of my Gaussian MLE = {accuracy}') 
    
    clf = KNearestNeighbor(X,y,10)
    pred = clf.predict(x_test)
    accuracy = clf.accuracy(pred,y_test)
    print(f'accuracy of K nearest neighbor = {accuracy}')
    
    
    clf = Multiclass_Logistic_Regression(learning_rate= 0.001,n_classes=2)
    clf.fit(X,y)
    pred = clf.predict(x_test)
    accuracy = clf.accuracy(pred,y_test)
    print(f'accuracy of my Multiclass Logisitic Regression = {accuracy}') 
    
    
    clf = LDA(n_classes=2,n_features = x_train.shape[1])
    clf.fit(X,counts)
    pred = clf.predict(x_test)
    accuracy = clf.accuracy(pred,y_test)
    print(f'accuracy of LDA = {accuracy}')
    
    clf = Multiclass_Gaussian_Naive_Bayes(x_train.shape[1],2)
    clf.fit(X,counts)
    pred = clf.predict(x_test)
    accuracy = clf.accuracy(pred,y_test)
    print(f'accuracy of Gaussian_Naive_Bayes = {accuracy}')
    
    
    clf = LogisticRegression()
    clf.fit(X,y)
    pred = clf.predict(x_test)
    accuracy = clf.accuracy(pred,y_test)
    print(f'accuracy of Logistic Regression = {accuracy}')
    
    
    

In [ ]:
## gaussian Kernel Density estimation from scipy
from scipy.stats import gaussian_kde

class0_kde = gaussian_kde(x0.T,bw_method='scott').evaluate(x_test.T)
class1_kde = gaussian_kde(x1.T,bw_method='scott').evaluate(x_test.T)
pred_kde_estimate = np.zeros(x_test.shape[0])

n_correct_preds = 0
for i in range(y_test.shape[0]):
    
    if class0_kde[i] > class1_kde[i]:
        pred_kde_estimate[i] = 0
    else:
        pred_kde_estimate[i] = 1
    if pred_kde_estimate[i] == y_test[i]:
        n_correct_preds += 1
        
accuracy = n_correct_preds/y_test.shape[0]
print(accuracy,pred_kde_estimate,y_test.shape[0])

In [ ]:
from scipy.stats import multivariate_normal

test_pdf0 = [np.mean(multivariate_normal.pdf(x0,x_test[i],np.identity(x0.shape[1]))) for i in range(x_test.shape[0])]
test_pdf1 = [np.mean(multivariate_normal.pdf(x1,x_test[i],np.identity(x1.shape[1]))) for i in range(x_test.shape[0])]

pred_Parzen = []
correct_preds = 0

for i in range(x_test.shape[0]):
    if test_pdf1[i]>test_pdf0[i]:
        pred_Parzen.append(1)
        if y_test[i] == 1:
            correct_preds += 1
    else:
        pred_Parzen.append(0)
        if y_test[i] == 0:
            correct_preds += 0
print(correct_preds,pred_Parzen)
